In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

# —— 1. Style & Font Settings —— #
plt.style.use('seaborn-v0_8-white')
plt.rcParams.update({
    # 'font.family':      'serif',   # Commonly used in academic papers
    'font.size':        16,        # Base font size 16pt
    'axes.titlesize':   18,        # Subplot title size
    'axes.labelsize':   17,        # Axis label size
    'legend.fontsize':  15,
    'xtick.labelsize':  14,
    'ytick.labelsize':  14,
    'axes.linewidth':   1.0,
    'lines.linewidth':  2.0,
    'grid.alpha':       0.3,
})

# —— 2. Unified Color Scheme —— #
color_SAASBO  = '#66C2A5'  # Teal
color_rand   = '#FC8D62'  # Coral Orange
color_LLM    = '#8DA0CB'  # Purple Blue
dot_color    = '#888888'  # Data Point Color

# —— 3. Process LLM Data (Only take the first 120 data points) —— #
llm_dir = "../LLM_case2"

# Configuration: Group folders belonging to the same run
# Each list represents a complete run, containing one or more folder names
# For example: If a single run is split into multiple folders (due to interruptions), put them in the same list
llm_folder_groups = [
    # Run 1: Possibly split into multiple folders
    ["Control_simulation_results_20251012_123102", "Control_simulation_results_20251012_182532"],
    # Run 2: Single folder
    ["Control_simulation_results_20251012_123105",'Control_simulation_results_20251012_182652'],
    # Run 3: Single folder
    ["Control_simulation_results_20251012_123111","Control_simulation_results_20251012_182532"],
    # More runs can be added...
    ["Control_simulation_results_20251014_132755"],
    ["Control_simulation_results_20251014_132800"],
    ["Control_simulation_results_20251014_132803"],
    ["Control_simulation_results_20251014_132804"],
    ["Control_simulation_results_20251014_132806"],
    ["Control_simulation_results_20251014_132808"],

]

llm_cumulative_mins = []
llm_best_results = []

# Process each group of folders (each group represents a complete run)
for folder_group in llm_folder_groups:
    combined_losses = []
    
    # Read and merge data from all folders in the group
    for folder_name in folder_group:
        folder_path = os.path.join(llm_dir, folder_name)
        log_path = os.path.join(folder_path, "generation_log.csv")
        
        if os.path.exists(log_path):
            df = pd.read_csv(log_path)
            if not df.empty:
                losses = df['Child_Loss'].values
                combined_losses.extend(losses)
            else:
                print(f"File {log_path} has no data.")
        else:
            print(f"Log file not found: {log_path}")
    
    # If data was successfully read
    if len(combined_losses) > 0:
        # Only take the first 120 data points (this number can be modified)
        combined_losses = np.array(combined_losses[:150])
        
        # Handle exceptionally large values (use the same threshold handling rules as SAASBO/Random)
        # If the first value >= threshold, set it to threshold
        if combined_losses[0] >= threshold:
            combined_losses[0] = threshold
        # For subsequent values, if >= threshold, set it to the previous value (maintain continuity)
        for i in range(1, len(combined_losses)):
            if combined_losses[i] >= threshold:
                combined_losses[i] = combined_losses[i-1]
        
        # Calculate cumulative minimum
        cumulative_min = np.minimum.accumulate(combined_losses)
        llm_cumulative_mins.append(cumulative_min)
        llm_best_results.append(cumulative_min[-1])
        print(f"Processed run with folders: {folder_group}, total points: {len(combined_losses)}")

# Convert to numpy arrays and pad
llm_best_results = np.array(llm_best_results)
if len(llm_cumulative_mins) > 0:
    max_len_llm = max(len(x) for x in llm_cumulative_mins)
    llm_cumulative_mins_padded = [np.pad(x, (0, max_len_llm - len(x)), 'edge') for x in llm_cumulative_mins]
    print(f"LLM: Found {len(llm_best_results)} runs, Mean = {np.mean(llm_best_results):.4f}, Std = {np.std(llm_best_results):.4f}")
else:
    print("Warning: No LLM data found!")
    llm_cumulative_mins_padded = []

# —— 4. Data Processing (Calculate Min and Max Values) —— #
bo_array    = np.array(bo_cumulative_mins_padded)
rand_array  = np.array(random_cumulative_mins_padded)
llm_array   = np.array(llm_cumulative_mins_padded)

bo_mean     = bo_array.mean(axis=0)
bo_min      = bo_array.min(axis=0)
bo_max      = bo_array.max(axis=0)
rand_mean   = rand_array.mean(axis=0)
rand_min    = rand_array.min(axis=0)
rand_max    = rand_array.max(axis=0)
llm_mean    = llm_array.mean(axis=0)
llm_min     = llm_array.min(axis=0)
llm_max     = llm_array.max(axis=0)

x           = np.arange(len(bo_mean))
x_llm       = np.arange(len(llm_mean))

# —— 5. Plotting —— #
fig, axes = plt.subplots(1, 2,
                         figsize=(9.5, 4.5),
                         sharey=False,
                         constrained_layout=True)

# —— a) Boxplot —— #
ax = axes[0]
box = ax.boxplot([bo_best_results, random_best_results, llm_best_results],
                 labels=['P2O(SAASBO)', 'P2O(Rnd)', 'P2P'],
                 patch_artist=True,
                 showfliers=False)

# Fill color & border
for patch, color in zip(box['boxes'], [color_SAASBO, color_rand, color_LLM]):
    patch.set_facecolor(color)
    patch.set_alpha(0.5)
    patch.set_edgecolor('black')
    patch.set_linewidth(1)
for key in ('whiskers','caps','medians'):
    for line in box[key]:
        line.set_color('black')
        line.set_linewidth(1)

# Add scatter points
for i, data in enumerate([bo_best_results, random_best_results, llm_best_results]):
    xs = np.random.normal(i+1, 0.04, size=len(data))
    ax.plot(xs, data, '.', color=dot_color, alpha=0.6, markersize=4)

ax.set_title('a', loc='left')
ax.set_xlabel('Method')
ax.set_ylabel('Best Total Loss')
# ax.set_ylim(0, 0.035)  # Set y-axis range
ax.grid(True, linestyle='--', linewidth=0.5)
# Simplify borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# —— b) Mean + Min-Max Range Curve —— #
ax = axes[1]
ax.plot(x, bo_mean,  label='P2O (SAASBO)',  color=color_SAASBO, marker='o', markersize=4)
ax.fill_between(x, bo_min, bo_max, color=color_SAASBO, alpha=0.2)
ax.plot(x, rand_mean, label='P2O (Random)', color=color_rand,  marker='x', markersize=5)
ax.fill_between(x, rand_min, rand_max, color=color_rand, alpha=0.2)
# Add LLM curve (multiple runs, including min-max range)
ax.plot(x_llm, llm_mean, label='P2P', color=color_LLM, marker='s', markersize=3)
ax.fill_between(x_llm, llm_min, llm_max, color=color_LLM, alpha=0.2)

ax.set_title('b', loc='left')
ax.set_xlabel('Trial Number')
ax.set_ylabel('Best-so-far Total Loss')
# ax.set_ylim(0, 0.02)  # Set y-axis range
# log
ax.set_yscale('log')

ax.legend(loc='upper right', frameon=False)
ax.grid(True, linestyle='--', linewidth=0.5)
# Simplify borders
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# —— 6. Save & Display —— #
plt.savefig("figure1_combined_SAASBO_vs_random_LLM.png", dpi=300)
plt.show()